In [2]:
%pip install weaviate-client python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [17]:
import json

def jprint(schema):
    print(json.dumps(schema, indent=4))

In [9]:
import os
import weaviate
from weaviate.util import generate_uuid5
from dotenv import load_dotenv

load_dotenv()

weaviate_api_key = os.environ['WEAVIATE_API_KEY']
weaviate_url = os.environ['WEAVIATE_URL']
openai_api_key = os.environ['OPENAI_API_KEY']

auth_config = weaviate.AuthApiKey(api_key=weaviate_api_key)

# set up the client
CLIENT = weaviate.Client(
    url=weaviate_url,
    auth_client_secret=auth_config,
    additional_headers={
        "X-OpenAI-Api-Key": openai_api_key
    }
)

In [11]:
class_name = "Work"
count = CLIENT.query.aggregate(class_name).with_meta_count().do()
print(count)

{'data': {'Aggregate': {'Work': [{'meta': {'count': 9995}}]}}}


In [12]:
nearText = {"concepts": ["art"]}
CLIENT.query.get(class_name="Work", properties=["title"]).with_limit(15).do()

{'data': {'Get': {'Work': [{'title': 'Santa Maria della Pace. Plan and renderings (with Baroque alterations)'},
    {'title': 'Tomb of Doge Niccolo Marchello (d. 1474). female virtue, architectural detail at left, from below left front'},
    {'title': 'Corcovado; Meditation; The Girl from Ipanema'},
    {'title': 'Armor. Helmets, their kinds and development during the centuries'},
    {'title': 'Alte Pinakothek. Interior'},
    {'title': 'Mens suits. One button Fairfax. Three button double breasted sack. The Cambria. The Hillhurst.'},
    {'title': 'Memo from Barry Olivier about priority list'},
    {'title': 'Johnny Rivers, Monterey Pop Festival'},
    {'title': 'Sacsayhuaman. view showing doorway from lower to second tier'},
    {'title': 'Tomb of Pope Innocent XII Pignatelli (b.Naples, 242nd Pope, r.1691-1700). General view from below right'},
    {'title': 'Information for U.C. - P.I.O. clip sheet'},
    {'title': 'Big Brother and the Holding Company, Monterey Pop Festival'},
    

In [15]:
CLIENT.query.get("Work", ["title", "subject"]).with_near_text({"concepts": ["exoplanetary orbits"]}).with_limit(10).with_additional(["certainty", "id"]).do()

{'data': {'Get': {'Work': [{'_additional': {'certainty': 0.8882298767566681,
      'id': '72548990-ddae-5831-889a-4eec44186844'},
     'subject': 'Painting',
     'title': "Jupiter, Mercury and 'Virtus'. Jupiter drawing"},
    {'_additional': {'certainty': 0.8881542086601257,
      'id': '2daca3ab-0714-5670-9ee0-73b01097d159'},
     'subject': 'Painting',
     'title': 'Apollo, Phaeton with Saturn and the Four Seasons. principal detail at upper right'},
    {'_additional': {'certainty': 0.8869164288043976,
      'id': '3e9acec7-3c9e-5577-bfc0-3ac807fe2494'},
     'subject': 'Painting|Sleep|Mythology, Classical|Apollo (Greek deity)|Muses (Greek deities)',
     'title': 'The Sleeping Apollo and the Muses'},
    {'_additional': {'certainty': 0.8861933946609497,
      'id': '43df8d57-01a6-59a3-9c95-a637381ee3c4'},
     'subject': 'Painting',
     'title': 'To JFK: a thousand roots did die with thee'},
    {'_additional': {'certainty': 0.8854801058769226,
      'id': '798cdc12-e665-5fcb-82b

In [18]:
max_groups = 10
max_objects_per_group = 3
response = (
    CLIENT.query
    .get("Work", ["title"])
    .with_near_text({
        "concepts": ["painting"]})
    .with_limit(10)
    .with_group_by(
        ["collection"],
        groups=max_groups,
        objects_per_group=max_objects_per_group
    )
    .with_additional([
        """
        group {
            id
            groupedBy {
                path
                value
            }
            count
            minDistance
            maxDistance
            hits {
                title
                identifier
                creator
                description
                subject
            }
        }
        """
    ])
    .do()
)

jprint(response)

{
    "data": {
        "Get": {
            "Work": [
                {
                    "_additional": {
                        "group": {
                            "count": 3,
                            "groupedBy": {
                                "path": [
                                    "collection"
                                ],
                                "value": "Saskia Art Images (CARLI), Records of Department of Art History"
                            },
                            "hits": [
                                {
                                    "creator": "Courbet, Gustave (French painter and designer, 1819-1877)",
                                    "description": null,
                                    "identifier": "4e7cbb5a-3d6d-4923-834c-e91dc9f6f851",
                                    "subject": "Painting",
                                    "title": "The Painter's Studio: A Real-Life-Allegory of the Last Seven Years of My Lif